<a href="https://colab.research.google.com/github/LAGISHETTYNANDITHA/aiml-2024/blob/main/2203a52097_aiml(a_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from copy import deepcopy
import numpy as np
import time

def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)

def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

def evaluvate_misplaced(puzzle, goal):
    steps = np.array([
        ('u', [0, 1, 2], -3),
        ('d', [6, 7, 8], 3),
        ('l', [0, 3, 6], -1),
        ('r', [2, 5, 8], 1)
    ], dtype=[('move', str, 1), ('position', list), ('head', int)])

    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    costg = coordinates(goal)

    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)

    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, hn)], dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)

        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)
        blank = int(np.where(puzzle == 0)[0])
        gn = gn + 1

        start_time = time.time()

        for s in steps:
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]

                already_in_state = any(np.array_equal(openstates, np.array(st['puzzle'])) for st in state)

                if not already_in_state:
                    end_time = time.time()
                    if end_time - start_time > 2:
                        print("The 8 puzzle is unsolvable\n")
                        return state, len(priority)

                    hn = misplaced_tiles(coordinates(openstates), costg)
                    new_state = np.array([(openstates.tolist(), position, gn, hn)], dtstate)
                    state = np.append(state, new_state)

                    fn = gn + hn
                    new_priority = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, new_priority)

                    if np.array_equal(openstates, goal):
                        print('The 8 puzzle is solvable\n')
                        return state, len(priority)

# Initial puzzle state
puzzle = [2, 8, 3,
          7, 1, 4,
          0, 6, 5]

# Goal state
goal = [1, 2, 3,
        8, 0, 4,
        7, 6, 5]

state, visited = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)

print(str(bestpath).replace('[', '').replace(']', ''))
totalmoves = len(bestpath) - 1
print('\nSteps to reach goal:', totalmoves)
visit = len(state) - visited
print('Total nodes visited:', visit, "\n")


The 8 puzzle is solvable

2 8 3
  7 1 4
  0 6 5

 2 8 3
  0 1 4
  7 6 5

 2 8 3
  1 0 4
  7 6 5

 2 0 3
  1 8 4
  7 6 5

 0 2 3
  1 8 4
  7 6 5

 1 2 3
  0 8 4
  7 6 5

 1 2 3
  8 0 4
  7 6 5

Steps to reach goal: 6
Total nodes visited: 11 



<ipython-input-3-f9d660d9182a>:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blank = int(np.where(puzzle == 0)[0])
